# <center/>MindInsight的模型溯源和数据溯源体验

## 概述
在AI训练的过程中，面对陌生的神经网络训练，经常需要事先优化神经网络训练中的参数，毕竟在训练一个十分复杂的神经网络时，有时候需要花费少则几天多则几周甚至更多的时间，为了更好的管理、调试和优化神经网络的训练过程，我们需要一个工具来对训练过程中的计算图、各种指标随着时间的变化趋势以及训练中使用到的图像信息进行分析和记录工作，而MindSpore就提供了一个对用户十分易用友好的可视化工具MindInsight，赋能给用户进行数据溯源和模型溯源的可视化分析，能明显提升用户对网络搭建过程和数据增强过程的纠错调优能力。而本次体验会从MindInsight的数据记录，可视化效果，如何方便用户在模型调优，数据调优上做一次整体流程的体验。

下面按照MindSpore的训练数据模型的正常步骤进行，当使用到MindInsight或者`SummaryCollector`算子进行数据保存操作时，会增加相应的说明，本次体验的整体流程如下：

1、数据集的准备，这里使用的是MNIST数据集。

2、构建一个网络，这里使用LeNet网络。(此处将使用第二种记录方式`ImageSummary`)。

3、训练网络和测试网络的搭建及运行。（此处将操作`SummaryCollector`初始化，并记录模型训练和模型测试相关信息）。

4、启动MindInsight服务。

5、模型溯源的使用。调整模型参数多次存储数据，并使用MindInsight的模型溯源功能对不同优化参数下训练产生的模型作对比，了解MindSpore中的各类优化对训练过程的影响及如何调优训练过程。

6、数据溯源的使用。调整数据参数多次存储数据，并使用MindInsight的数据溯源功能对不同数据集下训练产生的模型进行对比分析，了解如何调优。

本次体验将使用快速入门案例作为基础用例，将MindInsight的模型溯源和数据溯源的数据记录功能加入到案例中，快速入门案例的源码请参考：https://gitee.com/mindspore/docs/blob/r0.5/tutorials/tutorial_code/lenet.py 。

## 一、训练的数据集下载

### 1、数据集准备

#### 方法一：
从以下网址下载，并将数据包解压缩后放至Jupyter的工作目录下：<br/>训练数据集：{"http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz", "http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz"}
<br/>测试数据集：{"http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz", "http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz"}<br/>我们用下面代码查询jupyter的工作目录。

In [ ]:
import os
os.getcwd()

训练数据集放在----`Jupyter工作目录+\MNIST_Data\train\`，此时`train`文件夹内应该包含两个文件，`train-images-idx3-ubyte`和`train-labels-idx1-ubyte` <br/>测试数据集放在----`Jupyter工作目录+\MNIST_Data\test\`，此时`test`文件夹内应该包含两个文件，`t10k-images-idx3-ubyte`和`t10k-labels-idx1-ubyte`

#### 方法二：
直接执行下面代码，会自动进行训练集的下载与解压，但是整个过程根据网络好坏情况会需要花费几分钟时间。

In [ ]:
# Network request module, data download module, decompression module
import urllib.request   
from urllib.parse import urlparse
import gzip 

def unzipfile(gzip_path):
    """unzip dataset file
    Args:
        gzip_path: dataset file path
    """
    open_file = open(gzip_path.replace('.gz',''), 'wb')
    gz_file = gzip.GzipFile(gzip_path)
    open_file.write(gz_file.read())
    gz_file.close()
    
def download_dataset():
    """Download the dataset from http://yann.lecun.com/exdb/mnist/."""
    print("******Downloading the MNIST dataset******")
    train_path = "./MNIST_Data/train/" 
    test_path = "./MNIST_Data/test/"
    train_path_check = os.path.exists(train_path)
    test_path_check = os.path.exists(test_path)
    if train_path_check == False and test_path_check == False:
        os.makedirs(train_path)
        os.makedirs(test_path)
    train_url = {"http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz", "http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz"}
    test_url = {"http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz", "http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz"}
    
    for url in train_url:
        url_parse = urlparse(url)
        # split the file name from url
        file_name = os.path.join(train_path,url_parse.path.split('/')[-1])
        if not os.path.exists(file_name.replace('.gz', '')):
            file = urllib.request.urlretrieve(url, file_name)
            unzipfile(file_name)
            os.remove(file_name)
            
    for url in test_url:
        url_parse = urlparse(url)
        # split the file name from url
        file_name = os.path.join(test_path,url_parse.path.split('/')[-1])
        if not os.path.exists(file_name.replace('.gz', '')):
            file = urllib.request.urlretrieve(url, file_name)
            unzipfile(file_name)
            os.remove(file_name)

download_dataset()

这样就完成了数据集的下载解压缩工作。

### 2、数据集处理

数据集处理对于训练非常重要，好的数据集可以有效提高训练精度和效率。在加载数据集前，我们通常会对数据集进行一些处理。
<br/>我们定义一个函数`create_dataset`来创建数据集。在这个函数中，我们定义好需要进行的数据增强和处理操作：
<br/>1、定义数据集。
<br/>2、定义进行数据增强和处理所需要的一些参数。
<br/>3、根据参数，生成对应的数据增强操作。
<br/>4、使用`map`映射函数，将数据操作应用到数据集。
<br/>5、对生成的数据集进行处理。

> 具体的数据集操作可以在MindInsight的数据溯源中进行可视化分析。另外提取图像需要将`normalize`算子的数据处理(`CV.Rescale`)操作取消，否则取出来的图像为全黑图像。

In [ ]:
import mindspore.dataset.transforms.vision.c_transforms as CV
import mindspore.dataset.transforms.c_transforms as C
from mindspore.dataset.transforms.vision import Inter
from mindspore.common import dtype as mstype
import mindspore.dataset as ds

def create_dataset(data_path, batch_size=32, repeat_size=1,
                   num_parallel_workers=1):
    """ create dataset for train or test
    Args:
        data_path: Data path
        batch_size: The number of data records in each group
        repeat_size: The number of replicated data records
        num_parallel_workers: The number of parallel workers
    """
    # define dataset
    mnist_ds = ds.MnistDataset(data_path)

    # Define some parameters needed for data enhancement and rough justification
    resize_height, resize_width = 32, 32
    rescale = 1.0 / 255.0
    shift = 0.0
    rescale_nml = 1 / 0.3081
    shift_nml = -1 * 0.1307 / 0.3081

    # According to the parameters, generate the corresponding data enhancement method
    resize_op = CV.Resize((resize_height, resize_width), interpolation=Inter.LINEAR)  # Resize images to (32, 32) by bilinear interpolation
    rescale_nml_op = CV.Rescale(rescale_nml, shift_nml) # normalize images
    rescale_op = CV.Rescale(rescale, shift) # rescale images
    hwc2chw_op = CV.HWC2CHW() # change shape from (height, width, channel) to (channel, height, width) to fit network.
    type_cast_op = C.TypeCast(mstype.int32) # change data type of label to int32 to fit network

    # Using map() to apply operations to a dataset
    mnist_ds = mnist_ds.map(input_columns="label", operations=type_cast_op, num_parallel_workers=num_parallel_workers)
    mnist_ds = mnist_ds.map(input_columns="image", operations=resize_op, num_parallel_workers=num_parallel_workers)
    mnist_ds = mnist_ds.map(input_columns="image", operations=rescale_op, num_parallel_workers=num_parallel_workers)
    mnist_ds = mnist_ds.map(input_columns="image", operations=rescale_nml_op, num_parallel_workers=num_parallel_workers)
    mnist_ds = mnist_ds.map(input_columns="image", operations=hwc2chw_op, num_parallel_workers=num_parallel_workers)
    
    # Process the generated dataset
    buffer_size = 10000
    mnist_ds = mnist_ds.shuffle(buffer_size=buffer_size)  # 10000 as in LeNet train script
    mnist_ds = mnist_ds.batch(batch_size, drop_remainder=True)
    mnist_ds = mnist_ds.repeat(repeat_size)

    return mnist_ds

## 二、构建LeNet5网络

### 使用ImageSummary记录图像数据

在构建LeNet5网络的`__init__`中，初始化`ImageSummary`算子，同时在`construct`中将`ImageSummary`放在第一步，其第一个参数`image`为抽取出来的图片的自定义命名，第二个参数`x`是图像数据。此方法与`SummaryCollector`抽取图像的方法不冲突，可以同时使用。

In [1]:
from mindspore.ops import operations as P
import mindspore.nn as nn
from mindspore.common.initializer import TruncatedNormal

# Initialize 2D convolution function
def conv(in_channels, out_channels, kernel_size, stride=1, padding=0):
    """Conv layer weight initial."""
    weight = weight_variable()
    return nn.Conv2d(in_channels, out_channels,
                     kernel_size=kernel_size, stride=stride, padding=padding,
                     weight_init=weight, has_bias=False, pad_mode="valid")

# Initialize full connection layer
def fc_with_initialize(input_channels, out_channels):
    """Fc layer weight initial."""
    weight = weight_variable()
    bias = weight_variable()
    return nn.Dense(input_channels, out_channels, weight, bias)

# Set truncated normal distribution
def weight_variable():
    """Weight initial."""
    return TruncatedNormal(0.02)

class LeNet5(nn.Cell):
    """Lenet network structure."""
    # define the operator required
    def __init__(self):
        super(LeNet5, self).__init__()
        self.batch_size = 32 # 32 pictures in each group
        self.conv1 = conv(1, 6, 5) # Convolution layer 1, 1 channel input (1 Figure), 6 channel output (6 figures), convolution core 5 * 5
        self.conv2 = conv(6, 16, 5) # Convolution layer 2,6-channel input, 16 channel output, convolution kernel 5 * 5
        self.fc1 = fc_with_initialize(16 * 5 * 5, 120)
        self.fc2 = fc_with_initialize(120, 84)
        self.fc3 = fc_with_initialize(84, 10)
        self.relu = nn.ReLU()
        self.max_pool2d = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        #Init ImageSummary
        self.sm_image = P.ImageSummary()

    # use the preceding operators to construct networks
    def construct(self, x):
        self.sm_image("image",x)
        x = self.conv1(x) # 1*32*32-->6*28*28
        x = self.relu(x) # 6*28*28-->6*14*14
        x = self.max_pool2d(x) # Pool layer
        x = self.conv2(x) # Convolution layer
        x = self.relu(x) # Function excitation layer
        x = self.max_pool2d(x) # Pool layer
        x = self.flatten(x) # Dimensionality reduction
        x = self.fc1(x) # Full connection
        x = self.relu(x) # Function excitation layer
        x = self.fc2(x) # Full connection
        x = self.relu(x) # Function excitation layer
        x = self.fc3(x) # Full connection
        return x

## 三、训练网络和测试网络构建

### 1、使用SummaryCollector放入到训练网络中记录训练数据

`summary_callback`，即是`SummaryCollector`，在`model.train`的回调函数中使用，可以记录训练数据溯源和模型溯源信息。

In [ ]:
# Training and testing related modules
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig, LossMonitor, SummaryCollector, Callback
from mindspore.train import Model
import os


def train_net(model, epoch_size, mnist_path, repeat_size, ckpoint_cb, summary_collector):
    """Define the training method."""
    print("============== Starting Training ==============")
    # load training dataset
    ds_train = create_dataset(os.path.join(mnist_path, "train"), 32, repeat_size)
    model.train(epoch_size, ds_train, callbacks=[ckpoint_cb, LossMonitor(), summary_collector], dataset_sink_mode=True)

### 2、使用SummaryCollector放入到测试网络中记录测试数据

`summary_callback`，即是`SummaryCollector`，在`model.eval`的回调函数中使用，可以记录训练精度信息和测试样本数量信息。

In [3]:
from mindspore.train.serialization import load_checkpoint, load_param_into_net

def test_net(network, model, mnist_path, summary_collector):
    """Define the evaluation method."""
    print("============== Starting Testing ==============")
    # load the saved model for evaluation
    param_dict = load_checkpoint("checkpoint_lenet-3_1875.ckpt")
    # load parameter to the network
    load_param_into_net(network, param_dict)
    # load testing dataset
    ds_eval = create_dataset(os.path.join(mnist_path, "test"))
    acc = model.eval(ds_eval, callbacks=[summary_collector], dataset_sink_mode=True)
    print("============== Accuracy:{} ==============".format(acc))

### 3、主程序运行入口

初始化`SummaryCollector`，使用`collect_specified_data`控制需要记录的数据，我们这里只需要记录模型溯源和数据溯源，所以将`collect_train_lineage`和`collect_eval_lineage`参数设置成`True`,其他的参数使用`keep_default_action`设置成`False`，SummaryCollector能够记录哪些数据，请参考官网：<https://www.mindspore.cn/api/zh-CN/r0.5/api/python/mindspore/mindspore.train.html#mindspore.train.callback.SummaryCollector> 。

In [ ]:
from mindspore.train.callback import SummaryCollector
from mindspore.train.summary.summary_record import SummaryRecord
from mindspore.nn.metrics import Accuracy
from mindspore import context
from mindspore.nn.loss import SoftmaxCrossEntropyWithLogits

if __name__=="__main__":
    context.set_context(mode=context.GRAPH_MODE, device_target = "GPU")
    lr = 0.01 # learning rate
    momentum = 0.9 
    epoch_size = 3
    mnist_path = "./MNIST_Data"
    
    net_loss = SoftmaxCrossEntropyWithLogits(is_grad=False, sparse=True, reduction='mean')
    repeat_size = epoch_size
    # create the network
    network = LeNet5()

    # define the optimizer
    net_opt = nn.Momentum(network.trainable_params(), lr, momentum)
    config_ck = CheckpointConfig(save_checkpoint_steps=1875, keep_checkpoint_max=10)
    ckpoint_cb = ModelCheckpoint(prefix="checkpoint_lenet", config=config_ck)
    model = Model(network, net_loss, net_opt, metrics={"Accuracy": Accuracy()})
    
    collect_specified_data = {"collect_eval_lineage":True,"collect_train_lineage":True}
    summary_collector = SummaryCollector(summary_dir="./summary_base/quick_start_summary01", collect_specified_data=collect_specified_data, keep_default_action=False) 
    train_net(model, epoch_size, mnist_path, repeat_size, ckpoint_cb, summary_collector)
    test_net(network, model, mnist_path, summary_collector)

## 四、启动及关闭MindInsight服务

这里主要展示如何启用及关闭MindInsight，更多的命令集信息，请参考MindSpore官方网站：https://www.mindspore.cn/tutorial/zh-CN/r0.5/advanced_use/visualization_tutorials.html 。

- 启动MindInsight服务

    在安装过MindInsight的环境中启动MindInsight服务：
    - `--summary-base-dir`：MindInsight指定启动工作路径的命令。
    - `./summary_base`：SummaryRecord保存文件夹的目录。
    - `--port`：MindInsight指定启动的端口，数值可以任意为1~65535的范围内。

In [ ]:
import os
os.system("mindinsight start --summary-base-dir=./summary_base --port=8090")

查询是否启动成功，在网址输入:`127.0.0.1:8090`，如果看到如下界面说明启动成功。

![image](https://gitee.com/mindspore/docs/raw/master/tutorials/notebook/mindinsight/images/summary_list.png)

- 关闭MindInsight服务

    在安装过MindInsight的环境中输入命令：`mindinsight stop --port=8090`
    - `mindinsight stop`：MindInsight关闭服务命令。
    - `--port=8090`：即MindInsight服务开启在`8090`端口，所以这里写成`--port=8090`。

## 五、模型溯源

### 1、连接到模型溯源地址

浏览器中输入:`127.0.0.1:8090`，点击模型溯源，如下模型溯源界面：

![image](https://gitee.com/mindspore/docs/raw/master/tutorials/notebook/mindinsight/images/model_lineage_all.png)

我们可以勾选展示列，由于训练过程涉及的参数很多，在调整训练参数时，一般只会调整少量参数，所以对大部分相同参数可以去掉勾选，不显示出来，使得用户更方便的观察不同参数对模型训练的影响，上图中的不同参数的竖直线段代表的各个参数，数根连接各个参数的折线图代表不同的模型训练过程，其中各参数从左到右如下：

- `summary路径`：表示存储记录数据的文件夹路径，即`summary_dir`。
- `Accuracy`：模型的精度值。
- `loss`：模型的loss值。
- 网络：表示神经网络名称（用户可自行命名）。
- 优化器：表示训练过程中采用的优化器。
- 训练样本数量：训练样本数量。
- 测试样本数量：测试样本数量。
- 学习率：learning_rate的值。
- `epoch`：训练圈数。
- `steps`：训练步数。
- device数目：启用的训练卡数目。
- 模型大小：生成的模型文件`.ckpt`的大小。
- 损失函数：表示训练过程中采用的损失函数（用户可自行命名）。

根据上述记录的信息，我们可以调整模型训练过程中的参数，训练生成模型，然后选择要对比的训练，进行比对观察分析。

### 2、观察分析记录下来的溯源参数

下图选择了数条不同参数下训练生成的模型进行对比：

![image](https://gitee.com/mindspore/docs/raw/master/tutorials/notebook/mindinsight/images/model_lineage_cp.png)

在这几次训练的参数中，优化器，epoch和学习率都不一致，可以看到不同的训练生成的模型精度`Accuracy`和loss值是不一致的，当然最好是调整单个参数来观察对模型生成的影响，避免多重因素干扰，难以分辨哪个参数是正影响，哪个参数是负影响。这需要我们调整不同的参数，多训练几遍生成模型，分析各参数对训练产生的影响，这对前期学习AI训练时很有帮助。在以后应对复杂训练时，可以节省不少时间。
> 在多次训练时，需要将`summary_dir`的保存路径的文件夹进行重命名操作，否则训练记录的数据会生成在同一个文件夹下，而在同一文件夹下MindInsight只会读取最后一位数字比较大的文件即最后生成的文件。

## 六、数据溯源

### 1、连接到数据溯源地址

浏览器中输入：127.0.0.1:8090连接上MindInsight的服务，点击模型溯源，如下图数据溯源界面：

![image](https://gitee.com/mindspore/docs/raw/master/tutorials/notebook/mindinsight/images/data_lineage.png)

数据溯源的根本是重现数据集从左到右进行数据增强的整个过程，方便自己发现增强过程中是否有遗漏的步骤或者不合理的操作，方便自己查找错误，也方便自己找到最优的数据增强方式，毕竟一个好的数据集对模型训练是有事半功倍的效果的。

- `summary路径`：表示存储记录数据的文件夹名称，即为`SummaryRecord`的路径下的文件夹名称。
- `MnistDataset`：表示数据集信息，包含数据集路径。
- `Map_TypeCast`：定义数据集的类型。
- `Map_Resize`：图像缩放后的尺寸。
- `Map_Rescale`：图像的缩放比例。
- `Map_HWC2CHW`：数据集的张量由：高×宽×通道-->通道×高×宽。
- `Shuffle`：数据集混洗的缓存空间。
- `Batch`：每组训练样本数量。
- `Repeat`：数据图片复制次数，用于增强数据的数量。

### 2、观察分析数据溯源参数

可以从上图看到数据增强过程由原数据集MnistDataset开始，按照先后顺序经过了下面的操作：label的数据类型转换（`Map_Typecast`），图像的高宽缩放（`Map_Resize`），图像的比例缩放（`Map_Rescale`），图像数据的张量变换（`Map_HWC2CHW`），图像混洗（`Shuffle`），图像成组（`Batch`），图像数量增强（`Repeat`）然后输出训练需要的数据。显然这样的可视化的数据溯源功能，在你检查数据增强操作是否有误的时候，比起一行行的去检查代码效率多了。

## 最后关闭MindInsight服务

In [ ]:
import os
os.system("mindinsight stop --port=8090")

以上就是这次对MindInsight的使用方法，模型溯源和数据溯源的全部过程。